# Настойка ноутбука

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
# Расширить рабочее поле ноутбука на весь экран
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [65]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Описание ноутбука

В этом ноутбуке примедены методы Pandas для построения сводных таблиц, а также примеры их использования.

**Ресурсы:**
- [Pandas Doc: Изменение формы таблиц](https://pandas.pydata.org/docs/user_guide/reshaping.html)
- [Medium: Pivot_Tables](https://towardsdatascience.com/pandas-pivot-the-ultimate-guide-5c693e0771f3)

# Генерация данных

In [5]:
data = pd.DataFrame([
            ['16-10-2021', '23:56:10', 'A', 'a1', '0001', np.random.randint(100)], 
            ['16-10-2021', '23:58:30', 'A', 'a1', '0002', np.random.randint(100)],
            ['16-10-2021', '23:58:35', 'B', 'b1', '0003', np.random.randint(100)],
            ['16-10-2021', '23:58:56', 'B', 'b2', '0004', np.random.randint(100)],
            ['16-10-2021', '23:59:21', 'C', 'c1', '0005', np.random.randint(100)],
            ['16-10-2021', '23:59:42', 'C', 'c1', '0006', np.random.randint(100)],

            ['17-10-2021', '00:03:05', 'A', 'a1', '0007', np.random.randint(100)],
            ['17-10-2021', '00:03:25', 'A', 'a2', '0008', np.random.randint(100)],
            ['17-10-2021', '00:03:48', 'A', 'a3', '0009', np.random.randint(100)],
            ['17-10-2021', '00:05:24', 'B', 'b1', '0010', np.random.randint(100)],
            ['17-10-2021', '00:05:36', 'B', 'b1', '0011', np.random.randint(100)],
            ['17-10-2021', '00:05:44', 'B', 'b2', '0012', np.random.randint(100)],
            ['17-10-2021', '00:06:03', 'B', 'b2', '0013', np.random.randint(100)],
            ['17-10-2021', '03:12:13', 'B', 'b3', '0014', np.random.randint(100)],
            ['17-10-2021', '03:12:46', 'C', 'c1', '0015', np.random.randint(100)],
            ['17-10-2021', '13:06:54', 'C', 'c1', '0016', np.random.randint(100)], 
            ['17-10-2021', '13:12:10', 'C', 'c2', '0017', np.random.randint(100)], 
            ['17-10-2021', '19:48:30', 'C', 'c2', '0018', np.random.randint(100)], 

            ['18-10-2021', '00:06:05', 'C', 'c1', '0019', np.random.randint(100)], 
            ['18-10-2021', '03:08:03', 'C', 'c2', '0020', np.random.randint(100)],
            ['18-10-2021', '12:07:08', 'C', 'c3', '0021', np.random.randint(100)],
    
            ['19-10-2021', '00:04:03', 'A', 'a1', '0022', np.random.randint(100)], 
            ['19-10-2021', '02:08:20', 'A', 'a2', '0023', np.random.randint(100)],
            ['19-10-2021', '10:03:08', 'B', 'a2', '0024', np.random.randint(100)],
            ['19-10-2021', '12:07:08', 'B', 'b1', '0025', np.random.randint(100)],
    
], columns=['str_date', 'str_time','group', 'sub_group','val_id', 'value'])

data['datetime'] = data.apply(lambda row:  dt.datetime.strptime(f'{row.str_date} {row.str_time}', '%d-%m-%Y %H:%M:%S'), axis=1)

data['date'] = data.datetime.dt.date
data['time'] = data.datetime.dt.time
data['week'] = data.datetime.dt.isocalendar().week # Получить номер недели

# data['weekday'] = data.datetime.dt.strftime('%A') 
data['weekday'] = data.datetime.dt.day_name() # Получить день недели (на английском)

datetime_cols = ['datetime', 'date','time','week','weekday']
group_cols = ['group', 'sub_group']
data = data[[*datetime_cols, *group_cols, 'val_id', 'value']]

data.head()

,datetime,date,time,week,weekday,group,sub_group,val_id,value
0,2021-10-16 23:56:10,2021-10-16,23:56:10,41,Saturday,A,a1,0001,93
1,2021-10-16 23:58:30,2021-10-16,23:58:30,41,Saturday,A,a1,0002,11
2,2021-10-16 23:58:35,2021-10-16,23:58:35,41,Saturday,B,b1,0003,6
3,2021-10-16 23:58:56,2021-10-16,23:58:56,41,Saturday,B,b2,0004,77
4,2021-10-16 23:59:21,2021-10-16,23:59:21,41,Saturday,C,c1,0005,22


In [71]:
simple_data = pd.DataFrame({
    'datetime': pd.date_range('1/1/2000', periods=9, freq='min'),
    'group' : ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C'],
    'sub_group': ['a','a', 'a','b','a','c','a','b','c'],
    'value': [1,2,5,3,4,2,1,5,4],
    'val_id': ['1000','1001','1002','1003','1004','1005','1006','1007','1008']
})

simple_data

,datetime,group,sub_group,value,val_id
0,2000-01-01 00:00:00,A,a,1,1000
1,2000-01-01 00:01:00,A,a,2,1001
2,2000-01-01 00:02:00,A,a,5,1002
3,2000-01-01 00:03:00,B,b,3,1003
4,2000-01-01 00:04:00,B,a,4,1004
5,2000-01-01 00:05:00,B,c,2,1005
6,2000-01-01 00:06:00,C,a,1,1006
7,2000-01-01 00:07:00,C,b,5,1007
8,2000-01-01 00:08:00,C,c,4,1008


# Методы построения сводных таблиц

В Pandas, существует два метода для построения сводных таблиц:
1. pivot() - используется для разложения элементов в датафрейме по двум осям сводной таблицы. Не поддерживает агрегацию. При дублировании строк вызывает ошибку.
2. pivot_table() - используется для построения сводных таблиц. Поддерживает различные функции агрегации.

### [Метод pivot()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html)

В принципе pivot делает то же, что и метод unstack()

<img src="../../data/img/reshaping_unstack.png" width="500">

In [72]:
simple_data.set_index(["datetime", "sub_group"])["value"].unstack()

sub_group,a,b,c
datetime,,,
2000-01-01 00:00:00,1.0,NaN,NaN
2000-01-01 00:01:00,2.0,NaN,NaN
2000-01-01 00:02:00,5.0,NaN,NaN
2000-01-01 00:03:00,NaN,3.0,NaN
2000-01-01 00:04:00,4.0,NaN,NaN
2000-01-01 00:05:00,NaN,NaN,2.0
2000-01-01 00:06:00,1.0,NaN,NaN
2000-01-01 00:07:00,NaN,5.0,NaN
2000-01-01 00:08:00,NaN,NaN,4.0


In [73]:
df = simple_data
pv = df.pivot(index='datetime', columns='sub_group', values='value')
pv

sub_group,a,b,c
datetime,,,
2000-01-01 00:00:00,1.0,NaN,NaN
2000-01-01 00:01:00,2.0,NaN,NaN
2000-01-01 00:02:00,5.0,NaN,NaN
2000-01-01 00:03:00,NaN,3.0,NaN
2000-01-01 00:04:00,4.0,NaN,NaN
2000-01-01 00:05:00,NaN,NaN,2.0
2000-01-01 00:06:00,1.0,NaN,NaN
2000-01-01 00:07:00,NaN,5.0,NaN
2000-01-01 00:08:00,NaN,NaN,4.0


pivot ломается, если в индексах или колонках есть дубликаты

In [74]:
# simple_data с продублиованной последней строкой
wrong_simple_data = pd.concat([simple_data, pd.DataFrame(simple_data.iloc[-1]).T])
wrong_simple_data

,datetime,group,sub_group,value,val_id
0,2000-01-01 00:00:00,A,a,1,1000
1,2000-01-01 00:01:00,A,a,2,1001
2,2000-01-01 00:02:00,A,a,5,1002
3,2000-01-01 00:03:00,B,b,3,1003
4,2000-01-01 00:04:00,B,a,4,1004
5,2000-01-01 00:05:00,B,c,2,1005
6,2000-01-01 00:06:00,C,a,1,1006
7,2000-01-01 00:07:00,C,b,5,1007
8,2000-01-01 00:08:00,C,c,4,1008
8,2000-01-01 00:08:00,C,c,4,1008


In [75]:
df = wrong_simple_data
try:
    pv = df.pivot(index='group', columns='sub_group', values='value')
except ValueError as error:
    print(f"Возникает ошибка: {error}")

Возникает ошибка: Index contains duplicate entries, cannot reshape


## [Метод pivot_table()](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html)

In [76]:
df = wrong_simple_data
pv = df.pivot_table(index='group', columns='sub_group', values='value', aggfunc='nunique', fill_value=0)
pv

sub_group,a,b,c
group,,,
A,3,0,0
B,1,1,1
C,1,1,1


Механизм работы pivot_table больше напоминает эту команду. Здесь агрегирующая функция - это nunique. Но в принципе мы можем применить любую функцию

In [79]:
wrong_simple_data.groupby(['group', 'sub_group'])['value'].nunique().unstack().fillna(0).astype(int)

sub_group,a,b,c
group,,,
A,3,0,0
B,1,1,1
C,1,1,1


# Практики работы со сводными таблицами

## Вычисление суммы колонок и строк

In [80]:
simple_data.pivot_table(index='sub_group', columns='group', values='value', aggfunc='sum', 
                        margins=True, margins_name='Всего', 
                        fill_value=0)

group,A,B,C,Всего
sub_group,,,,
a,8,4,1,13
b,0,3,5,8
c,0,2,4,6
Всего,8,9,10,27


## Сворачивание сводной таблицы 

<img src="../../data/img/reshaping_stack.png" width="500">

In [81]:
df = simple_data
pv = df.pivot_table(index='group', columns='sub_group', values='value', aggfunc='nunique', fill_value=0)
pv

sub_group,a,b,c
group,,,
A,3,0,0
B,1,1,1
C,1,1,1


In [82]:
pv.stack().to_frame()

0
group sub_group   
A     a          3
      b          0
      c          0
B     a          1
      b          1
      c          1
C     a          1
      b          1
      c          1

## Рассчет процентной сводной таблицы

In [83]:
df = data
pv_cnt = df.pivot_table(index='weekday', columns='group', values='value', aggfunc='nunique', fill_value=0)
pv_cnt

group,A,B,C
weekday,,,
Monday,0,0,3
Saturday,2,2,2
Sunday,3,5,4
Tuesday,2,2,0


In [84]:
pv_pct = (pv_cnt.div(pv_cnt.sum(axis=1), axis=0) * 100).round().astype(int)
pv_pct

group,A,B,C
weekday,,,
Monday,0,0,100
Saturday,33,33,33
Sunday,25,42,33
Tuesday,50,50,0


## Многомерные сводные таблицы

In [85]:
df = data

# Применение нескольких агрегирующих функций
pv = df.pivot_table(
    index=['week', 'weekday'], 
    columns=['group', 'sub_group'], 
    values = 'value',
    aggfunc=['sum', 'mean'],
    fill_value=0,
)

pv.head()

sum                                          mean              \
group            A           B                  C              A               
sub_group       a1  a2  a3  a2   b1   b2  b3   c1   c2  c3    a1    a2    a3   
week weekday                                                                   
41   Saturday  104   0   0   0    6   77   0   72    0   0  52.0   0.0   0.0   
     Sunday     91  40  36   0  104  155  37  130  139   0  91.0  40.0  36.0   
42   Monday      0   0   0   0    0    0   0   12   57  25   0.0   0.0   0.0   
     Tuesday    45   7   0  96    0    0   0    0    0   0  45.0   7.0   0.0   

                                                         
group             B                       C              
sub_group        a2    b1    b2    b3    c1    c2    c3  
week weekday                                             
41   Saturday   0.0   6.0  77.0   0.0  36.0   0.0   0.0  
     Sunday     0.0  52.0  77.5  37.0  65.0  69.5   0.0  
42   Monday     0.0   0.0   0.0   0.0  12.0  57.0  25.0  
     Tuesday   96.0   0.0   0.0   0.0   0.0   0.0   0.0

In [86]:
df = data

# Применение разных функций к разным значениям
pv = df.pivot_table(
    index=['week', 'weekday'], 
    columns=['group', 'sub_group'], 
    #          col  | func
    aggfunc={'value': 'sum', 'val_id': 'nunique'},
    fill_value=0,
)

pv

val_id                            value                        \
group              A        B           C           A           B             
sub_group         a1 a2 a3 a2 b1 b2 b3 c1 c2 c3    a1  a2  a3  a2   b1   b2   
week weekday                                                                  
41   Saturday      2  0  0  0  1  1  0  2  0  0   104   0   0   0    6   77   
     Sunday        1  1  1  0  2  2  1  2  2  0    91  40  36   0  104  155   
42   Monday        0  0  0  0  0  0  0  1  1  1     0   0   0   0    0    0   
     Tuesday       1  1  0  1  1  0  0  0  0  0    45   7   0  96    0    0   

                                 
group                C           
sub_group      b3   c1   c2  c3  
week weekday                     
41   Saturday   0   72    0   0  
     Sunday    37  130  139   0  
42   Monday     0   12   57  25  
     Tuesday    0    0    0   0

## Кастомные бины (интервалы) и автоматические квантили

In [87]:
simple_data['values_custom_bins'] = pd.cut(simple_data['value'], [0, 3, 5])
simple_data['auto_values_quantiles'] = pd.qcut(simple_data['value'], 2)
simple_data[['value', 'values_custom_bins', 'auto_values_quantiles', 'group','val_id']]

,value,values_custom_bins,auto_values_quantiles,group,val_id
0,1,"(0, 3]","(0.999, 3.0]",A,1000
1,2,"(0, 3]","(0.999, 3.0]",A,1001
2,5,"(3, 5]","(3.0, 5.0]",A,1002
3,3,"(0, 3]","(0.999, 3.0]",B,1003
4,4,"(3, 5]","(3.0, 5.0]",B,1004
5,2,"(0, 3]","(0.999, 3.0]",B,1005
6,1,"(0, 3]","(0.999, 3.0]",C,1006
7,5,"(3, 5]","(3.0, 5.0]",C,1007
8,4,"(3, 5]","(3.0, 5.0]",C,1008


In [88]:
df = simple_data

pv = df.pivot_table(index='values_custom_bins', columns='group', values='val_id', aggfunc='nunique')
pv

group,A,B,C
values_custom_bins,,,
"(0, 3]",2,2,1
"(3, 5]",1,1,2


In [89]:
df = simple_data

pv = df.pivot_table(index='auto_values_quantiles', columns='group', values='val_id', aggfunc='nunique')
pv

group,A,B,C
auto_values_quantiles,,,
"(0.999, 3.0]",2,2,1
"(3.0, 5.0]",1,1,2


## Временные бины

In [90]:
data['time_diff'] = data.datetime.diff().fillna(pd.Timedelta(0))
data.head()

,datetime,date,time,week,weekday,group,sub_group,val_id,value,time_diff
0,2021-10-16 23:56:10,2021-10-16,23:56:10,41,Saturday,A,a1,0001,93,0 days 00:00:00
1,2021-10-16 23:58:30,2021-10-16,23:58:30,41,Saturday,A,a1,0002,11,0 days 00:02:20
2,2021-10-16 23:58:35,2021-10-16,23:58:35,41,Saturday,B,b1,0003,6,0 days 00:00:05
3,2021-10-16 23:58:56,2021-10-16,23:58:56,41,Saturday,B,b2,0004,77,0 days 00:00:21
4,2021-10-16 23:59:21,2021-10-16,23:59:21,41,Saturday,C,c1,0005,22,0 days 00:00:25


In [94]:
time_bins = [
    pd.Timedelta(seconds = 0),
    pd.Timedelta(seconds = 30),
    pd.Timedelta(minutes = 1),
    pd.Timedelta(minutes = 5),
    pd.Timedelta(hours = 1),
    pd.Timedelta(hours = 2),
    pd.Timedelta(hours = 4),
    pd.Timedelta(hours = 6),
    pd.Timedelta(hours = 12),
    pd.Timedelta(hours = 18),
    pd.Timedelta(hours = 23),
    pd.Timedelta(days = 2),
    pd.Timedelta(days = 100)
]

labels = ['0-30seconds','30seconds-1minute', 
          '1-5minutes', '5minutes-1hours', 
          '1-2hours', '2-4hours', '4-6hours', '6-12hours', '12-18hours', '18-23hours', 
          '1-2 days', '2+ days']

simple_data['custom_timediff_bins'] = pd.cut(data['time_diff'], time_bins, labels = labels)
simple_data

,datetime,group,sub_group,value,val_id,values_custom_bins,auto_values_quantiles,custom_timediff_bins
0,2000-01-01 00:00:00,A,a,1,1000,"(0, 3]","(0.999, 3.0]",NaN
1,2000-01-01 00:01:00,A,a,2,1001,"(0, 3]","(0.999, 3.0]",1-5minutes
2,2000-01-01 00:02:00,A,a,5,1002,"(3, 5]","(3.0, 5.0]",0-30seconds
3,2000-01-01 00:03:00,B,b,3,1003,"(0, 3]","(0.999, 3.0]",0-30seconds
4,2000-01-01 00:04:00,B,a,4,1004,"(3, 5]","(3.0, 5.0]",0-30seconds
5,2000-01-01 00:05:00,B,c,2,1005,"(0, 3]","(0.999, 3.0]",0-30seconds
6,2000-01-01 00:06:00,C,a,1,1006,"(0, 3]","(0.999, 3.0]",1-5minutes
7,2000-01-01 00:07:00,C,b,5,1007,"(3, 5]","(3.0, 5.0]",0-30seconds
8,2000-01-01 00:08:00,C,c,4,1008,"(3, 5]","(3.0, 5.0]",0-30seconds


In [96]:
df = simple_data
pv = df.pivot_table(index='custom_timediff_bins', columns='group', values='val_id', aggfunc='nunique')
pv

group,A,B,C
custom_timediff_bins,,,
0-30seconds,1,3,2
30seconds-1minute,0,0,0
1-5minutes,1,0,1
5minutes-1hours,0,0,0
1-2hours,0,0,0
2-4hours,0,0,0
4-6hours,0,0,0
6-12hours,0,0,0
12-18hours,0,0,0


## Как посчитать значения значения за определенный временной период?

### Метод resample() 

В этом примере функция работает странно. Строки считаются неверно. А если предварительно заполнить пустые значения нулями - ответ меняется

In [97]:
df = simple_data.set_index('datetime')
df = df.pivot_table(index='datetime', columns='sub_group', values='val_id', aggfunc='nunique', fill_value=0)
df

sub_group,a,b,c
datetime,,,
2000-01-01 00:00:00,1,0,0
2000-01-01 00:01:00,1,0,0
2000-01-01 00:02:00,1,0,0
2000-01-01 00:03:00,0,1,0
2000-01-01 00:04:00,1,0,0
2000-01-01 00:05:00,0,0,1
2000-01-01 00:06:00,1,0,0
2000-01-01 00:07:00,0,1,0
2000-01-01 00:08:00,0,0,1


In [98]:
tmp_count = df.resample('3min').sum()
tmp_count

sub_group,a,b,c
datetime,,,
2000-01-01 00:00:00,3,0,0
2000-01-01 00:03:00,1,1,1
2000-01-01 00:06:00,1,1,1


#### В этом случае сохранится название индекса. Это можно исправить

In [99]:
tmp_count = tmp_count.reset_index()
tmp_count

sub_group,datetime,a,b,c
0,2000-01-01 00:00:00,3,0,0
1,2000-01-01 00:03:00,1,1,1
2,2000-01-01 00:06:00,1,1,1


In [100]:
tmp_count = tmp_count.rename_axis(None).rename_axis(None, axis=1)
tmp_count

,datetime,a,b,c
0,2000-01-01 00:00:00,3,0,0
1,2000-01-01 00:03:00,1,1,1
2,2000-01-01 00:06:00,1,1,1


In [101]:
sub_groups = sorted(simple_data.sub_group.unique())
tmp_count['date'] = tmp_count.datetime.dt.date
tmp_count['time'] = tmp_count.datetime.dt.strftime('%H:%M')
tmp_count['all'] = tmp_count[sub_groups].sum(axis=1)

tmp_count[['date','time','all', *sub_groups]]

,date,time,all,a,b,c
0,2000-01-01,00:00,3,3,0,0
1,2000-01-01,00:03,3,1,1,1
2,2000-01-01,00:06,3,1,1,1


### К сводным таблицам можно добавить heatmap

In [102]:
# Depricated
# tmp_count_styled = (
#     tmp_count[['date','time','all', *sub_groups]]
#             .style
#             .format({col: "{}" for col in tmp_count.columns})
#             .background_gradient(
#                 subset=sub_groups, # По каким колонкам считать heatmap 
#                 axis=1, # 0 - считать по колонкам, 1 - считать по строкам
#                 cmap='Greys' # выбор цвета
#              )
# )

# tmp_count_styled